In [2]:
from bs4 import BeautifulSoup as bs
import lxml
import pandas as pd

In [78]:
with open("UC 3/etl_generated/R21-11/SWS_CANInterface_012/zz_generated/AUTOSAR_SWS_CANInterface_resolved.arxml", "r") as file:
    handlerA = file.read()

with open("UC 3/etl_generated/R22-11/SWS_CANInterface_012/zz_generated/AUTOSAR_SWS_CANInterface_resolved.arxml", "r") as file:
    handlerB = file.read()

soupA = bs(handlerA, "lxml-xml")
list_of_all_idsA = soupA.find_all("SHORT-NAME")
soupB = bs(handlerB, "lxml-xml")
list_of_all_idsB = soupB.find_all("SHORT-NAME")

list_of_all_idsA = [id.getText() for id in list_of_all_idsA]
list_of_all_idsB = [id.getText() for id in list_of_all_idsB]

dfA = pd.DataFrame({"idsA": list_of_all_idsA})
dfB = pd.DataFrame({"idsB": list_of_all_idsB})
df_merged = pd.merge(dfA, dfB, how = "outer", left_on = "idsA", right_on = "idsB")
df_merged["type"] = "unchanged"
df_merged["type"][df_merged.idsB.isna()] = "deleted"
df_merged["type"][df_merged.idsA.isna()] = "new"
df_merged.head()

,idsA,idsB,type
0,SWS_CANIF_00378,SWS_CANIF_00378,unchanged
1,SWS_CANIF_00672,SWS_CANIF_00672,unchanged
2,SWS_CANIF_00903,SWS_CANIF_00903,unchanged
3,SWS_CANIF_00023,SWS_CANIF_00023,unchanged
4,SWS_CANIF_00662,SWS_CANIF_00662,unchanged


# New Part: Change Detection

Retrieve all strings for given SHORT-NAME

In [102]:
df = df_merged.copy()
df['Text Release A']=""
df['Text Release B']=""
soup = soupA

for row in df.iterrows():
    if (soup.find('SHORT-NAME', string=row[1][0])) and (not pd.isna(row[1][0])):
        df['Text Release A'].iloc[row[0]] = [x.getText() for x in soup.find('SHORT-NAME', string=row[1][0]).parent.find_all('L-1')]    

soup = soupB
for row in df.iterrows():
    if (soup.find('SHORT-NAME', string=row[1][1])) and (not pd.isna(row[1][1])):
        df['Text Release B'].iloc[row[0]] = [x.getText() for x in soup.find('SHORT-NAME', string=row[1][1]).parent.find_all('L-1')]
df

,idsA,idsB,type,Text Release A,Text Release B
0,SWS_CANIF_00378,SWS_CANIF_00378,unchanged,[\nCanIf shall access the location of the API ...,[\nCanIf shall access the location of the API ...
1,SWS_CANIF_00672,SWS_CANIF_00672,unchanged,[The header file CanIf.h only contains extern ...,[The header file CanIf.h only contains extern ...
2,SWS_CANIF_00903,SWS_CANIF_00903,unchanged,[\nCanIf shall include the header file Mirror....,[\nCanIf shall include the header file Mirror....
3,SWS_CANIF_00023,SWS_CANIF_00023,unchanged,[\nCanIf shall avoid direct access to hardware...,[\nCanIf shall avoid direct access to hardware...
4,SWS_CANIF_00662,SWS_CANIF_00662,unchanged,[\nCanIf shall use two types of HOH to enable ...,[\nCanIf shall use two types of HOH to enable ...
...,...,...,...,...,...
508,NaN,ECUC_CanIf_00860,new,,"[\nSWS Item\n, \nECUC_CanIf_00860\n, \nContain..."
509,NaN,ECUC_CanIf_00864,new,,"[\nSWS Item\n, \nECUC_CanIf_00864\n, \nParamet..."
510,NaN,ECUC_CanIf_00861,new,,"[\nSWS Item\n, \nECUC_CanIf_00861\n, \nParamet..."
511,NaN,ECUC_CanIf_00863,new,,"[\nSWS Item\n, \nECUC_CanIf_00863\n, \nParamet..."


Example IDs:

SWS_CANIF_00378 --> no change in content
SWS_CANIF_00851 --> content changed
ECUC_CanIf_00619 --> content only slightly changed, but different structure

In [157]:
df[df['idsA']=='SWS_CANIF_00142']['Text Release A'].values[0]

['\nModule\n',
 '\nHeader File\n',
 '\nImported Type\n',
 'Can',
 'Can_GeneralTypes.h',
 '\nCan_ControllerStateType\n',
 'Can_GeneralTypes.h',
 '\nCan_ErrorStateType\n',
 'Can_GeneralTypes.h',
 '\nCan_ErrorType\n',
 'Can_GeneralTypes.h',
 '\nCan_HwHandleType\n',
 'Can_GeneralTypes.h',
 '\nCan_HwType\n',
 'Can_GeneralTypes.h',
 '\nCan_IdType\n',
 'Can_GeneralTypes.h',
 '\nCan_PduType\n',
 'Can_GeneralTypes.h',
 '\nCan_TimeStampType (draft)',
 'CanTrcv',
 'Can_GeneralTypes.h',
 '\nCanTrcv_TrcvModeType\n',
 'Can_GeneralTypes.h',
 '\nCanTrcv_TrcvWakeupModeType\n',
 'Can_GeneralTypes.h',
 '\nCanTrcv_TrcvWakeupReasonType\n',
 'ComStack_Types',
 'ComStack_Types.h',
 '\nPduIdType\n',
 'ComStack_Types.h',
 '\nPduInfoType\n',
 'ComStack_Types.h',
 '\nPduLengthType\n',
 'EcuM',
 'EcuM.h',
 '\nEcuM_WakeupSourceType\n',
 'IdsM',
 'IdsM_Types.h',
 '\nIdsM_SecurityEventIdType\n',
 'Std',
 'Std_Types.h',
 '\nStd_ReturnType\n',
 'Std_Types.h',
 '\nStd_VersionInfoType\n']

In [159]:
df[df['idsB']=='SWS_CANIF_00142']['Text Release B'].values[0]

['\nModule\n',
 '\nHeader File\n',
 '\nImported Type\n',
 'Can',
 'Can_GeneralTypes.h',
 '\nCan_ControllerStateType\n',
 'Can_GeneralTypes.h',
 '\nCan_ErrorStateType\n',
 'Can_GeneralTypes.h',
 '\nCan_ErrorType\n',
 'Can_GeneralTypes.h',
 '\nCan_HwHandleType\n',
 'Can_GeneralTypes.h',
 '\nCan_HwType\n',
 'Can_GeneralTypes.h',
 '\nCan_IdType\n',
 'Can_GeneralTypes.h',
 '\nCan_PduType\n',
 'Can_GeneralTypes.h',
 '\nCan_TimeStampType (draft)',
 'CanTrcv',
 'Can_GeneralTypes.h',
 '\nCanTrcv_TrcvModeType\n',
 'Can_GeneralTypes.h',
 '\nCanTrcv_TrcvWakeupModeType\n',
 'Can_GeneralTypes.h',
 '\nCanTrcv_TrcvWakeupReasonType\n',
 'CanXL',
 'Can_GeneralTypes.h',
 '\nCanXL_HwType\n',
 'Can_GeneralTypes.h',
 '\nCanXL_Params\n',
 'Can_GeneralTypes.h',
 '\nCanXL_PduType\n',
 'ComStack_Types',
 'ComStack_Types.h',
 '\nPduIdType\n',
 'ComStack_Types.h',
 '\nPduInfoType\n',
 'ComStack_Types.h',
 '\nPduLengthType\n',
 'EcuM',
 'EcuM.h',
 '\nEcuM_WakeupSourceType\n',
 'IdsM',
 'IdsM_Types.h',
 '\nIdsM_Sec

In [154]:
df_merged_2 = df.copy()
df_merged_2 = df_merged_2[df_merged_2['type']=='unchanged']
df_merged_2['Difference in String'] = (df_merged_2['Text Release A'] != df_merged_2['Text Release B'])

no_changed = df_merged_2['Difference in String'][df_merged_2['Difference in String']==True].count()
no_total = df_merged_2[df_merged_2['type']=='unchanged']['Difference in String'].count()

In [160]:
no_changed/no_total

0.2623655913978495

In [156]:
df_merged_change = df_merged_2[df_merged_2['Difference in String']==True] 
df_merged_change = df_merged_change[(df_merged_change['idsA'].str.contains('SWS', na=False, regex=False)) | (df_merged_change['idsB'].str.contains('SWS', na=False, regex=False))] 
df_merged_change

,idsA,idsB,type,Text Release A,Text Release B,Difference in String
55,SWS_CANIF_00851,SWS_CANIF_00851,unchanged,[If MetaData is configured for a received L-SD...,[If MetaData item CAN_ID_32 is configured for ...,True
63,SWS_CANIF_00677,SWS_CANIF_00677,unchanged,[If a controller mode referenced by Controller...,[If a controller mode referenced by Controller...,True
120,SWS_CANIF_00162,SWS_CANIF_00162,unchanged,[If the call of Can_Write() returns E_OK the t...,[If the call of Can_Write()/CanXL_Write() retu...,True
123,SWS_CANIF_00893,SWS_CANIF_00893,unchanged,[When CanIf_Transmit() is called with PduInfoP...,[When CanIf_Transmit() is called with PduInfoP...,True
293,SWS_CANIF_91006,SWS_CANIF_91006,unchanged,"[\nType of error\n, \nRelated error code\n, \n...","[\nType of error\n, \nRelated error code\n, \n...",True
294,SWS_CANIF_91007,SWS_CANIF_91007,unchanged,"[\nType of error\n, \nRelated error code\n, \n...","[\nType of error\n, \nRelated error code\n, \n...",True
295,SWS_CANIF_00142,SWS_CANIF_00142,unchanged,"[\nModule\n, \nHeader File\n, \nImported Type\...","[\nModule\n, \nHeader File\n, \nImported Type\...",True
341,SWS_CANIF_00294,SWS_CANIF_00294,unchanged,"[\nAPI Function\n, \nHeader File\n, \nDescript...","[\nAPI Function\n, \nHeader File\n, \nDescript...",True
